In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
import scattertext as st
from scipy.sparse.linalg import svds
import numpy as np
import os
import spacy
from pprint import pprint

In [2]:
pdf_dir = "../TEMAC/sota-papers/"
files = os.listdir(pdf_dir)

In [4]:
(len(files), files)

(20,
 ['wang2018.pdf',
  'mormont2018.pdf',
  'serrano2018.pdf',
  'PID5546301.pdf',
  'yang2017.pdf',
  'xia2017.pdf',
  'Cao_Partial_Transfer_Learning_CVPR_2018_paper.pdf',
  'oliveira2018.pdf',
  'aygun2017.pdf',
  'lucena2017.pdf',
  'ge2017.pdf',
  'montescondori2017.pdf',
  'zia2017.pdf',
  'mathew2017.pdf',
  'barbosakloss2018.pdf',
  'zamir2018.pdf',
  '10.1007@978-3-319-62410-515.pdf',
  'rezende2017.pdf',
  'venkateswara2017.pdf',
  '65982.pdf'])

In [5]:
txt_dir = "./sota20papers_txt/"
have = set(os.listdir(txt_dir))

In [6]:
for i,f in enumerate(files): 
    txt_basename = f + '.txt'
    if txt_basename in have:
        print('%d/%d skipping %s, already exists.' % (i, len(files), txt_basename, ))
        continue
    pdf_path = os.path.join(pdf_dir, f)
    txt_path = os.path.join(txt_dir, txt_basename)
    cmd = "pdftotext %s %s" % (pdf_path, txt_path)
    os.system(cmd)
    print('%d/%d %s' % (i, len(files), cmd))

0/20 skipping wang2018.pdf.txt, already exists.
1/20 skipping mormont2018.pdf.txt, already exists.
2/20 skipping serrano2018.pdf.txt, already exists.
3/20 skipping PID5546301.pdf.txt, already exists.
4/20 skipping yang2017.pdf.txt, already exists.
5/20 skipping xia2017.pdf.txt, already exists.
6/20 skipping Cao_Partial_Transfer_Learning_CVPR_2018_paper.pdf.txt, already exists.
7/20 skipping oliveira2018.pdf.txt, already exists.
8/20 skipping aygun2017.pdf.txt, already exists.
9/20 skipping lucena2017.pdf.txt, already exists.
10/20 skipping ge2017.pdf.txt, already exists.
11/20 skipping montescondori2017.pdf.txt, already exists.
12/20 skipping zia2017.pdf.txt, already exists.
13/20 skipping mathew2017.pdf.txt, already exists.
14/20 skipping barbosakloss2018.pdf.txt, already exists.
15/20 skipping zamir2018.pdf.txt, already exists.
16/20 skipping 10.1007@978-3-319-62410-515.pdf.txt, already exists.
17/20 skipping rezende2017.pdf.txt, already exists.
18/20 skipping venkateswara2017.pdf.tx

In [7]:
txt_files = os.listdir(txt_dir); (len(txt_files), txt_files)

(20,
 ['zia2017.pdf.txt',
  '10.1007@978-3-319-62410-515.pdf.txt',
  'lucena2017.pdf.txt',
  'ge2017.pdf.txt',
  'zamir2018.pdf.txt',
  'montescondori2017.pdf.txt',
  'oliveira2018.pdf.txt',
  'PID5546301.pdf.txt',
  'mormont2018.pdf.txt',
  'serrano2018.pdf.txt',
  'wang2018.pdf.txt',
  'mathew2017.pdf.txt',
  'barbosakloss2018.pdf.txt',
  'venkateswara2017.pdf.txt',
  '65982.pdf.txt',
  'xia2017.pdf.txt',
  'yang2017.pdf.txt',
  'rezende2017.pdf.txt',
  'Cao_Partial_Transfer_Learning_CVPR_2018_paper.pdf.txt',
  'aygun2017.pdf.txt'])

In [8]:
data = pd.DataFrame(txt_files); data.head(3)

,0
0,zia2017.pdf.txt
1,10.1007@978-3-319-62410-515.pdf.txt
2,lucena2017.pdf.txt


In [9]:
data.columns=['Arquivo']
data['Categoria']="FRONTIER"

In [10]:
data['Texto']=data["Arquivo"].apply(lambda fname: open("./sota20abstracts_txt/"+fname, "r").read().replace('\n', ''))

In [11]:
data.head(3)

,Arquivo,Categoria,Texto
0,zia2017.pdf.txt,FRONTIER,RGB-D Object Recognition Using Deep Convolutio...
1,10.1007@978-3-319-62410-515.pdf.txt,FRONTIER,Deep neural networks and transfer learning app...
2,lucena2017.pdf.txt,FRONTIER,Transfer Learning Using ConvolutionalNeural Ne...


In [12]:
top20 = os.listdir("top20abstracts_txt/"); (len(top20), top20)

(20,
 ['10_1109--TKDE_2009_191.pdf.txt',
  '10_1109--TPAMI_2015_2437384.pdf.txt',
  '10_1109--TNN_2010_2091281.pdf.txt',
  '10_1109--TPAMI_2013_167.pdf.txt',
  '10_1109--ICCV_2013_274.pdf.txt',
  '10_1109--TPAMI_2015_2408354.pdf.txt',
  '10_1109--TPAMI_2016_2599174.pdf.txt',
  'taylor_rl_survey.pdf.txt',
  '10_1007--s11263-014-0703-y.pdf.txt',
  '10_1109--TPAMI_2016_2572683.pdf.txt',
  '10_1109--TIP_2016_2516952.pdf.txt',
  '10_1109--ICCV_2015_336.pdf.txt',
  '10_1109--ICCV_2015_73.pdf.txt',
  '10_1007--s10994-009-5152-4.pdf.txt',
  '10_1016--j_knosys_2015_01_010.pdf.txt',
  '10_1109--TNNLS_2014_2330900.pdf.txt',
  '10_1007--978-3-319-10578-9_13.pdf.txt',
  '10_1109--TPAMI_2009_57.pdf.txt',
  '10_1109--TPAMI_2011_114.pdf.txt',
  '10_1109--TKDE_2013_111.pdf.txt'])

In [13]:
t = pd.DataFrame(top20)

In [14]:
t.columns=['Arquivo']
t['Categoria']="TOP"
t.head(3)

,Arquivo,Categoria
0,10_1109--TKDE_2009_191.pdf.txt,TOP
1,10_1109--TPAMI_2015_2437384.pdf.txt,TOP
2,10_1109--TNN_2010_2091281.pdf.txt,TOP


In [15]:
t['Texto']=t["Arquivo"].apply(lambda fname: open("./top20abstracts_txt/"+fname, "r").read().replace('\n', ''))

In [16]:
data = data.append(t, ignore_index = True)

In [17]:
data

,Arquivo,Categoria,Texto
0,zia2017.pdf.txt,FRONTIER,RGB-D Object Recognition Using Deep Convolutio...
1,10.1007@978-3-319-62410-515.pdf.txt,FRONTIER,Deep neural networks and transfer learning app...
2,lucena2017.pdf.txt,FRONTIER,Transfer Learning Using ConvolutionalNeural Ne...
3,ge2017.pdf.txt,FRONTIER,Borrowing Treasures from the Wealthy: Deep Tra...
4,zamir2018.pdf.txt,FRONTIER,Taskonomy: Disentangling Task Transfer Learnin...
5,montescondori2017.pdf.txt,FRONTIER,Comparison between traditional texture methods...
6,oliveira2018.pdf.txt,FRONTIER,Deep Transfer Learning for Segmentation ofAnat...
7,PID5546301.pdf.txt,FRONTIER,Delaunay Triangulation Data Augmentation guide...
8,mormont2018.pdf.txt,FRONTIER,Comparison of deep transfer learning strategie...
9,serrano2018.pdf.txt,FRONTIER,Automated Detection of Hummingbirdsin Images: ...


In [18]:
nlp = spacy.load('en')

In [19]:
corpus = (st.CorpusFromPandas(data,
                              category_col='Categoria',
                              text_col='Texto',
                              nlp=st.whitespace_nlp_with_sentences)
          .build()
          .get_unigram_corpus()
          .compact(st.ClassPercentageCompactor(term_count=2,
                                               term_ranker=st.OncePerDocFrequencyRanker)))

In [20]:
html = st.produce_characteristic_explorer(
    corpus,
    category="FRONTIER",
    category_name="Frontier",
    not_category_name="Classic",
    width_in_pixels=500,
    save_svg_button=True
    
    
)

In [39]:
open("Frontier.html", 'wb').write(html.encode('utf-8'))

453588

In [23]:
html = st.produce_scattertext_explorer(corpus,
        category='FRONTIER',
        category_name='Frontier',
        not_category_name='Classic',
        width_in_pixels=400,
        save_svg_button=True)

In [24]:
open("Frontier3.html", 'wb').write(html.encode('utf-8'))

404174